In [ ]:
!python --version

Python 3.6.9


In [ ]:
!wget https://www.python.org/ftp/python/3.10.1/Python-3.10.1tgz
!tar xvfz Python-3.10.1tgz
!Python-3.10.1/configure
!make
!sudo make install

In [ ]:
!python --version

In [ ]:
!pip install --upgrade mxnet>=1.6.0
!pip install gluonnlp==0.9.1
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [ ]:
!pip install onnxruntime==1.3.0

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import mxnet
import gluonnlp as nlp

In [ ]:
#transformers 
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#kobert
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Chatbot_Data = pd.read_csv('/content/drive/MyDrive/T3Q-자연어/CHATBOT/new_data.csv', index_col = 0)
Chatbot_Data

In [ ]:
Chatbot_Data[['label', 'raw_label']].nunique()

In [ ]:
Emotion = Chatbot_Data[['label', 'raw_label']].drop_duplicates().sort_values('label')
Emotion_data = {label: emotion for label, emotion in zip(Emotion['label'], Emotion['raw_label'])}

Emotion_data

In [ ]:
dataset_train, dataset_test = train_test_split(Chatbot_Data, 
                                               test_size = 0.4, 
                                               random_state = 0, 
                                               stratify = Chatbot_Data.label)

In [ ]:
dataset_val, dataset_test = train_test_split(dataset_test, 
                                               test_size = 0.5, 
                                               random_state = 0, 
                                               stratify = dataset_test.label)

In [ ]:
dataset_train.raw_label.value_counts()

In [ ]:
dataset_val.raw_label.value_counts()

In [ ]:
dataset_test.raw_label.value_counts()

#### KoBERT 입력 데이터 만들기

In [ ]:
import kobert_tokenizer
MODEL_NAME = 'skt/kobert-base-v1'
bert_model = BertModel.from_pretrained(MODEL_NAME, return_dict=True)
kobert_tokenizer = KoBERTTokenizer.from_pretrained(MODEL_NAME)


In [ ]:
result = kobert_tokenizer.tokenize('너는 내년 대선 때 투표할 수 있어?')
print(result)
print([kobert_tokenizer.encode(token) for token in result])

In [ ]:
kobert_tokenizer.encode(result)

In [ ]:
kobert_vocab = kobert_tokenizer.get_vocab()
kobert_vocab

In [ ]:
for i in dataset_train.Q:
    vocab = kobert_tokenizer(i + '13').input_ids
    vocab.append(13)
    print(vocab)
    break

In [ ]:
for i in dataset_train:
    print(i[0])
    break

In [ ]:
for i in dataset_train.label:
    print(np.int32(i))

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, 
                dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.sentences =  [self.tokenizer(i[0] + str(i[2])).input_ids for i in self.dataset]
        self.labels = [np.int32(i) for i in self.label]

    def __len__(self):
        return len(self.label)

    def __getitem__(self, index:int):
        return self.sentences[index]

In [ ]:
bert_train = BERTDataset(dataset_train, kobert_tokenizer)

In [ ]:
bert_train[1]

In [ ]:
bert_dataloader = torch.utils.data.DataLoader(bert_train, batch_size=32, shuffle = False, 
                                              drop_last = True)
for i in bert_dataloader:
    print(i)
    break


In [ ]:
dataset_train

In [ ]:
for i in dataset_train.Q:
    print(i)

In [ ]:
## setting parameters

max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
tok = nlp.data.BERTSPTokenizer(tokenizer, lower=False)
